In [ ]:
from sage.misc.converting_dict import KeyConvertingDict
from sage.misc.sage_timeit import sage_timeit
from time import time
from sage.matrix.constructor import Matrix

def get_safe_prime(bits):    
    while True:
        q = random_prime(2^bits)
        p = 2*q + 1        
        if is_prime(p) and p.ndigits(2)==bits:
            return p

def generate_factor_base(smooth):
    primes =[]
    tmp = 2
    while tmp < smooth:
        primes.append(tmp)
        tmp = next_prime(tmp)        
    return primes


def generate_row(primes,factors): 
    i=0; j=0; length_i=len(primes); length_j=len(factors); row=[]    
    while i< length_i:
        if j>=length_j:
            row=row+([0]*(length_i-i))
            break
        if primes[i]==factors[j][0]:
            row.append(factors[j][1])
            j=j+1
        else:
            row.append(0)
        i=i+1
    return row

          
def calculate_factor_base(g,p,primes,m,double=false,flag_g=false):   
#    m = matrix(0,len(primes)+1) 
#   
#    if double and flag_g:
#        global m_g
#        m = m_g
#    elif double and not flag_g:
#        global m_b
#        m = m_b        
#        
 
    F = FiniteField(p)
    g_order = g.multiplicative_order()
    rows = 0
    flag = false 
    factor_g_order=factor(g_order) 
    while true:        
        t = F.random_element()
        while g^t==1:
            print("random number is needed to regenerate")
            t = F.random_element()   
        f = factor(int(g^t))        
        if f[-1][0]<=primes[-1]:
            row = generate_row(primes,f)
            row.append(int(t))
            #m = m.stack(vector(row))
            m.append(row)
            rows = rows+1
            flag = true            
        
        if flag and rows%len(primes)==0:
            flag = false            
            matrixs = []
            for base, exponent in factor_g_order: 
                t = matrix(m)
                tmp = t[:,:-1].change_ring(Zmod(base^exponent))
                #tmp = m[:,:-1].change_ring(Zmod(base^exponent))
                list_log = []
                try:
                    list_log = tmp.solve_right(vector(t[:,-1]))
                except ValueError as e:
                    print(e)
                    print(tmp)
                    print(m[:,-1])
                    list_log = [0]*len(primes)
                matrixs.append(list_log) 
                
            list_log =[int(i) for i in  matrixs[0]]
            modulu_1 = int(factor_g_order[0][0]^factor_g_order[0][1]) 
            for i in range(1,len(matrixs)):
                modulu_2 = int(factor_g_order[i][0]^factor_g_order[i][1])
                for j in range(len(matrixs[i])):
                    r_1=int(list_log[j])
                    r_2=int(matrixs[i][j])
                    t =  crt(r_1,r_2,modulu_1,modulu_2)    #不能合并为一行
                    list_log[j] = t
                modulu_1 = modulu_1*modulu_2
                     
            logs_part = []
            primes_part = []
            for i in range(len(list_log)):
                if g^int(list_log[i])==int(primes[i]):  
                    logs_part.append(list_log[i])
                    primes_part.append(primes[i])
            if double:
#                if flag_g:                    
#                    m_g = m
#                else:                   
#                    m_b = m   
                return logs_part, primes_part
            
            elif len(logs_part) == len(primes):                  
                return list_log
            
def calculate_target_index(g,b,p,primes,list_log,):       
    F = FiniteField(p)
    row = [] 
    t=0 
    while true:            
        t = F.random_element()
        while g^t==1:
            print("random number is needed to regenerate")
            t = F.random_element() 
        f = factor(int(b*g^t))        
        if f[-1][0]<=primes[-1]:
            row = generate_row(primes,f)
            break
    x = (vector(row)*vector(list_log)-int(t)) % g.multiplicative_order()    
    return x  



def index_calculus(g,b,p,smooth):    
    primes = generate_factor_base(smooth)    
    m = []
    list_log = calculate_factor_base(g,p,primes,m)
    #print("m_0:",m)
    target_x = calculate_target_index(g,b,p,primes,list_log) 

    if g^target_x!=b:
        print("calculated target_x in index calculat method is wrong, please check codes\n")
    return target_x

        

##################################################################################        
##################            my own method                      #################
##################################################################################
#因为增加了检查代码，我的算法时间复杂度变高了，后面需要清理一下，让实现更加高效
##################################################################################



def double_calculate_factor_base(g,p,b,primes): 
#    global m_g
    m_g = matrix(0,len(primes)+1) 
#    global m_b
    m_b = matrix(0,len(primes)+1)
    intersection = set()
    time_1 = 0
    time_2 = 0
    #loop = 0
    m_g = []
    m_b = []
    while true:
        #loop = loop+1
        #print("loop:",loop)
        rt = cputime(subprocesses=True) 
        logs_g, g_primes = calculate_factor_base(g,p,primes,m_g,double=true,flag_g=true)
        time_1 = time_1+float(cputime(subprocesses=True) - rt)
        
        rt = cputime(subprocesses=True)
        logs_b, b_primes = calculate_factor_base(b,p,primes,m_b,double=true,flag_g=false)
        time_2 = time_2+float(cputime(subprocesses=True) - rt)
        #print("len(primes:",len(primes))
        #print(len(g_primes))
        #print(len(b_primes))
        intersection = set(g_primes).intersection(set(b_primes))
        if len(intersection)!=0:
                break
                
    #print("m_g:",m_g)
    #print("m_b:",m_b)
    prime = intersection.pop()
    index = g_primes.index(prime)
    s =logs_g[index] 
    index = b_primes.index(prime)
    t =logs_b[index]
    return s,t,prime,min(time_1,time_2)
    
    
def double_calculate_target_index(g_index,b_index,p,g):
    g_index = int(g_index)
    b_index = int(b_index)
    g_order = g.multiplicative_order()
    w,s,t = xgcd(b_index,g_order)
    s = s*g_index/w % g_order
    div = gcd(b_index,g_order)
    for i in range(div):
        if g^s==b:            
            break
        s = (s+g_order/div)% g_order           
    return s  
     
    

def double_index_calculus(g,b,p,smooth):
    primes = generate_factor_base(smooth)
    g_index,b_index,prime, min_time = double_calculate_factor_base(g,p,b,primes)     
    c_x = double_calculate_target_index(g_index,b_index,p,g)
    if g^c_x!=b:
        print("g_index,g^g_index,b_index,b^b_index,prime,g,b,p:",g_index,g^g_index,b_index,b^b_index,prime,g,b,p)
        print("calculated c_x in double index calculat method is wrong, please check codes\n")
    return c_x, min_time


##################################################################################       
print("###################################")

set_random_seed(int(time()))
bits = list(range(30,100,10)) 
print("bits:",bits)

time_list_1 = [0]*len(bits)
time_list_2 = [0]*len(bits)
for i in range(len(bits)):
    loop = 40
    tmp_time_1 = 0
    tmp_time_2 = 0
    B_i = 0
    B_d = 0
    for j in range(loop):
        if j%5==0:
            print("{}/{}".format(j,loop))
        p = get_safe_prime(bits[i])
        F = FiniteField(p)
        g = F.multiplicative_generator()     
        x = F.random_element()
        b = g^x
        
        t = log(p).n(5)        
        B_i = ceil(e^sqrt(t*log(t)/4))
        
        rt = cputime(subprocesses=True)
        c_x = index_calculus(g,b,p,B_i)
        tmp_time_1 = tmp_time_1+float(cputime(subprocesses=True) - rt)
        
        B_d = ceil(e^sqrt(t*log(t)/2))
        rt = cputime(subprocesses=True)
        c_x, mintime = double_index_calculus(g,b,p,B_d)
        tmp_time_2 = tmp_time_2+float(cputime(subprocesses=True) - rt - mintime)
    
    average = tmp_time_1/loop
    print("index calculus algorithm:")
    print("smooth,bits,time:",B_i,bits[i],average)
    time_list_1.append((bits[i],B_i,average))        
    save(time_list_1, "time_list_1")

    average = tmp_time_2/loop
    print("double index calculus algorithm:")
    print("smooth,bits,time:",B_d,bits[i],average)
    time_list_2.append((bits[i],B_d,average))
    save(time_list_2,"time_list_2")
                           
                           

time_list_1 = load("./time_list_1")
time_list_2 = load("./time_list_2")
  
points_1 = []
for i in range(len(time_list_1)):
    tmp = time_list_1[i]
    if tmp[2]<10:
        points_1.append(tmp)
        if tmp[2]<0:
            print(tmp[2],"negative")

points_2 = []
for i in range(len(time_list_2)):
    tmp = time_list_2[i]
    if tmp[2]<10:
        points_2.append(tmp)
        if tmp[2]<0:
            print(tmp[2],"negative")            
            

cm = colormaps.rainbow
cf = lambda x, y: ((x+y)/2) 

fig = list_plot3d(points_1,  color="red", size=10, marker='o', alpha=10, aspect_ratio=(1,0.01,1),frame=false)
fig = fig + list_plot3d(points_2,  color="blue", size=10, marker='o', alpha=10, aspect_ratio=(1,0.01,1),frame=false)
fig = fig+text3d("smooth boundary",(34,500,-2),color="black",fontsize=20,fontweight='bold')  
fig = fig+text3d("bits",(24,990,-2),color="black",fontsize=20,fontweight='bold')   
fig = fig+text3d("time (s)",(37,-100,2),color="black",fontsize=20,fontweight='bold')    
fig.show()






#print("###################################")


#global m_g
#global m_b


#set_random_seed(int(time()))         
##set_random_seed(100) 
##smoothes = list(range(10,30,1)) + list(range(30,1000,20))
##smoothes = list(range(10,100,10)) + list(range(100,1000,100)) 
#smoothes = list(range(100,1000,100))
#print("smooth:",smoothes)
#bits = list(range(20,45,5)) 
#print("bits:",bits)

#time_list_1 = []
#time_list_2 = []
#for i in range(len(bits)):
#    for k in range(len(smoothes)):
#        
#        p = get_safe_prime(bits[i])
#        F = FiniteField(p)
#        g = F.multiplicative_generator()     
#        x = F.random_element()
#        b = g^x
#
#        tmp = 0
#        loop =2000
#        for count in range(loop):
#            rt = cputime(subprocesses=True)
#            c_x = index_calculus(g,b,p,smoothes[k])
#            tmp = tmp+float(cputime(subprocesses=True) - rt)
#        average = tmp/loop
#        print("smooth,bits,time:",smoothes[k],bits[i],average)
#        time_list_1.append((bits[i],smoothes[k],average))        
#        save(time_list_1, "time_list_1")
#                           
#        tmp = 0
#        for count in range(loop):
#            rt = cputime(subprocesses=True)
#            c_x, mintime = double_index_calculus(g,b,p,smoothes[k])
#            tmp = tmp+float(cputime(subprocesses=True) - rt - mintime)
#        average = tmp/loop
#        print("smooth,bits,time:",smoothes[k],bits[i],average)
#        time_list_2.append((bits[i],smoothes[k],average))
#        save(time_list_2,"time_list_2")
#                           
#                           
#        #tmp = sage_timeit("", globals(), repeat=1)
#        #print("smooth,bits,time:",smoothes[k],bits[i],mean(tmp.series))
#        #time_list_2.append((bits[i],smoothes[k],mean(tmp.series)))
#        #save(time_list_2,"time_list_2")
#
#time_list_1 = load("./time_list_1")
#time_list_2 = load("./time_list_2")
#  
#points_1 = []
#for i in range(len(time_list_1)):
#    tmp = time_list_1[i]
#    if tmp[2]<10:
#        points_1.append(tmp)
#        if tmp[2]<0:
#            print(tmp[2],"negative")
#
#points_2 = []
#for i in range(len(time_list_2)):
#    tmp = time_list_2[i]
#    if tmp[2]<10:
#        points_2.append(tmp)
#        if tmp[2]<0:
#            print(tmp[2],"negative")            
#            

#cm = colormaps.rainbow
#cf = lambda x, y: ((x+y)/2) 

#fig = list_plot3d(points_1,  color="red", size=10, marker='o', alpha=10, aspect_ratio=(1,0.01,1),frame=false)
#fig = fig + list_plot3d(points_2,  color="blue", size=10, marker='o', alpha=10, aspect_ratio=(1,0.01,1),frame=false)
#fig = fig+text3d("smooth boundary",(34,500,-2),color="black",fontsize=20,fontweight='bold')  
#fig = fig+text3d("bits",(24,990,-2),color="black",fontsize=20,fontweight='bold')   
#fig = fig+text3d("time (s)",(37,-100,2),color="black",fontsize=20,fontweight='bold')    
#fig.show()










###################################
bits: [30, 40, 50, 60, 70, 80, 90]
0/40
5/40
10/40
15/40
20/40
25/40
30/40
35/40
index calculus algorithm:
smooth,bits,time: 54 30 0.15972500000000026
double index calculus algorithm:
smooth,bits,time: 320 30 0.060575000000000226
0/40
5/40
10/40
15/40
20/40
25/40
30/40
35/40
index calculus algorithm:
smooth,bits,time: 152 40 2.733125
double index calculus algorithm:
smooth,bits,time: 1088 40 1.0331750000000002
0/40
5/40
10/40


In [24]:
t = crt(1,2,5,9)
print(545800*455515%831646)

346946


In [7]:
w,s,t =  xgcd(70184,897467)
print(w,s,t)


1 44257 -3461
